In [37]:
from collections import OrderedDict
from datetime import datetime
import logging
import os
from pprint import pprint

import geojson
import geopandas as gpd
import fiona
import matplotlib as plt
import pandas as pd
import pyodbc
from shapely.geometry import (
    LineString, mapping, shape
)
from shapely.ops import (
    split as shapely_split, transform
)

from python_gis.poc.landgrid_processor import LandgridProcessor
from python_gis.poc.io.pgsql import PgWriter
from python_gis.poc.io.mssql import MsWriter
from python_gis.poc.io.shapefile import (
    read_shapefile
)
from python_gis.poc.tools.spatial import (
    remove_holes, fix_anti_meridian
)
import python_gis.poc.util.log_config
from python_gis.poc.util.sql_config import (
    TX_BLOCKS
)

logger = logging.getLogger(__name__)

In [ ]:
!ls $DIML_HOME

In [2]:
# Ms Sql
ms_driver = os.environ["MSSQL_DRIVER"]
ms_host = os.environ["MSSQL_SERVER"]
ms_db = os.environ["MSSQL_DATABASE"]
ms_user = os.environ["MSSQL_UID"]
ms_pwd = os.environ["MSSQL_PWD"]

# Postgis
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]

# WGS84 (epsg:4326)
gdb_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')
ddl_path = os.path.join(os.environ["DIML_HOME"], 'database', 'mssql', 'schema.sql')
out_path = os.path.join(os.environ["DATA_DIR"], 'shapefile_out')
idx_path = os.path.join(os.environ["DIML_HOME"], 'database', "indexes.sql")
test_path = os.path.join(os.environ["DATA_DIR"], 'mssql_test')

## Load Blocks

In [5]:
%%time

tx_path = os.path.join(out_path, f'Texas_Abstracts.shp')
tx_df = gpd.read_file(tx_path)

# TODO: May need to exclude NULL Township as it results in large
#  MultiPolygons. Check against reference (TWP: NULL, BLK: 1).
tx_df.dropna(how='all', subset=['Township', 'Block'],
             inplace=True)

tx_df.loc[tx_df.Township.isna(), ['Township']] = 'Missing'
tx_df.loc[tx_df.Block.isna(), ['Block']] = 'Missing'

CPU times: user 13.5 s, sys: 580 ms, total: 14.1 s
Wall time: 14.1 s


,PERIMETER,FIPS,CountyName,Shape_Leng,Shape_Area,AbstractNu,AbstractNa,Block,Township,Section,Abstract_1,FormNumber,ControlNum,geometry
0,0.051277,033,Borden,0.051277,0.000161,986,H&TC RR CO,97,None,316,"POLK, T T",None,None,"POLYGON ((-101.32458 32.78959, -101.32486 32.7..."
1,0.063304,033,Borden,0.063305,0.000248,270,T&P RR CO,31,5N,27,None,None,None,"POLYGON ((-101.48972 32.76752, -101.49429 32.7..."
2,0.051023,033,Borden,0.051023,0.000161,1160,H&TC RR CO,97,None,273,"HILL, D S",None,None,"POLYGON ((-101.32486 32.77504, -101.32504 32.7..."
3,0.046116,033,Borden,0.046115,0.000123,1341,H&TC RR CO,97,None,424,"ROBERTSON, W D",None,None,"POLYGON ((-101.21085 32.85421, -101.21072 32.8..."
4,0.063737,033,Borden,0.063738,0.000252,1228,H&TC RR CO,97,None,409,"REA, C L",None,None,"POLYGON ((-101.20281 32.83202, -101.22017 32.8..."


In [9]:
tx_df.loc[:, ['CountyName', 'Township', 'Block', 'geometry']]

,CountyName,Township,Block,geometry
0,Borden,Missing,97,"POLYGON ((-101.32458 32.78959, -101.32486 32.7..."
1,Borden,5N,31,"POLYGON ((-101.48972 32.76752, -101.49429 32.7..."
2,Borden,Missing,97,"POLYGON ((-101.32486 32.77504, -101.32504 32.7..."
3,Borden,Missing,97,"POLYGON ((-101.21085 32.85421, -101.21072 32.8..."
4,Borden,Missing,97,"POLYGON ((-101.20281 32.83202, -101.22017 32.8..."
...,...,...,...,...
310684,Pecos,9S,50,"POLYGON ((-103.30410 30.95254, -103.30502 30.9..."
310685,Pecos,Missing,1,"POLYGON ((-103.35540 30.93789, -103.36374 30.9..."
310686,Pecos,10S,50,"POLYGON ((-103.27870 30.93816, -103.27875 30.9..."
310687,Pecos,Missing,11,"POLYGON ((-103.47314 30.85763, -103.48239 30.8..."


## Create Data set

In [15]:
%%time

tx_blocks_df = (tx_df.loc[:, ['Township', 'Block', 'geometry']]  # .copy()
                .dissolve(by=['Township', 'Block'], aggfunc='first')
                .reset_index()
                )

tx_blocks_df.head()

CPU times: user 16.7 s, sys: 4 ms, total: 16.7 s
Wall time: 16.7 s


,Township,Block,geometry
0,-13,C-13,"POLYGON ((-103.99030 31.03204, -103.99030 31.0..."
1,1,54,"POLYGON ((-103.66624 31.88345, -103.67545 31.8..."
2,1,55,"POLYGON ((-103.70140 31.89790, -103.70160 31.8..."
3,1,56,"POLYGON ((-103.84755 31.88344, -103.84907 31.8..."
4,1,57,"POLYGON ((-103.90759 31.89829, -103.90770 31.8..."


## Multi-Polygon to Rows (Method 1)

In [41]:
tx_block_1_df = tx_blocks_df.copy()
tx_block_1_df['geom_type'] = tx_block_1_df.geometry.geom_type


In [43]:
tx_block_1_df = tx_block_1_df.loc[tx_blocks_df.geom_type == 'MultiPolygon', :].copy()
tx_block_1_df.head()

,Township,Block,geometry,geom_type
13,1,78,"MULTIPOLYGON (((-106.19180 31.88349, -106.2006...",MultiPolygon
14,1,79,"MULTIPOLYGON (((-106.25194 31.82622, -106.2517...",MultiPolygon
16,1,81,"MULTIPOLYGON (((-106.42302 31.86312, -106.4232...",MultiPolygon
17,1,82,"MULTIPOLYGON (((-106.52890 31.91314, -106.5359...",MultiPolygon
18,10,48,"MULTIPOLYGON (((-102.56688 30.25972, -102.5721...",MultiPolygon


In [44]:
tx_block_1_df['geom_list'] = [g for g in tx_block_1_df.geometry]

In [45]:
tx_block_1_df.head()

,Township,Block,geometry,geom_type,geom_list
13,1,78,"MULTIPOLYGON (((-106.19180 31.88349, -106.2006...",MultiPolygon,"(POLYGON ((-106.191802222 31.88349144100005, -..."
14,1,79,"MULTIPOLYGON (((-106.25194 31.82622, -106.2517...",MultiPolygon,"(POLYGON ((-106.251938357 31.82622162100006, -..."
16,1,81,"MULTIPOLYGON (((-106.42302 31.86312, -106.4232...",MultiPolygon,(POLYGON ((-106.4230167869999 31.8631242650000...
17,1,82,"MULTIPOLYGON (((-106.52890 31.91314, -106.5359...",MultiPolygon,"(POLYGON ((-106.528903858 31.91313742100004, -..."
18,10,48,"MULTIPOLYGON (((-102.56688 30.25972, -102.5721...",MultiPolygon,"(POLYGON ((-102.566882766 30.25971651200007, -..."


In [46]:
tx_block_1_df['geom_list'].apply(pd.Series)

,0,1,2,3,4,5,6,7,8,9,...,582,583,584,585,586,587,588,589,590,591
13,"POLYGON ((-106.191802222 31.88349144100005, -1...","POLYGON ((-106.191373993 31.82376458800007, -1...","POLYGON ((-106.116162882 31.88287412000005, -1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,"POLYGON ((-106.251938357 31.82622162100006, -1...",POLYGON ((-106.2006129869999 31.88353904400003...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,POLYGON ((-106.4230167869999 31.86312426500007...,POLYGON ((-106.4742752929999 31.88429779100005...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,"POLYGON ((-106.528903858 31.91313742100004, -1...","POLYGON ((-106.528738027 31.98540340300008, -1...",POLYGON ((-106.5116119369999 31.95673094900008...,POLYGON ((-106.5287722959999 31.92766900600003...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,"POLYGON ((-102.566882766 30.25971651200007, -1...","POLYGON ((-103.16912323 30.83711843300006, -10...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436,"POLYGON ((-100.036185014 30.29973186600006, -1...","POLYGON ((-102.88771062 30.46852259200006, -10...","POLYGON ((-101.22654399 30.44279074500002, -10...",POLYGON ((-101.1107378309999 30.44037750000007...,POLYGON ((-100.8151896959999 33.65417380000002...,POLYGON ((-100.8589986969999 33.68304632000007...,POLYGON ((-100.8853485719999 33.69033267100008...,"POLYGON ((-100.869040797 33.71940630300003, -1...","POLYGON ((-100.937520631 34.29239430600006, -1...",POLYGON ((-100.5615970969999 33.25820970700005...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439,POLYGON ((-98.85504866599996 28.82831960700003...,POLYGON ((-98.85711847599998 28.79982115000007...,POLYGON ((-98.89564601299998 28.83813717100003...,"POLYGON ((-102.316003998 31.26429920300006, -1...","POLYGON ((-102.283704326 31.27198837500003, -1...","POLYGON ((-101.882994797 30.10596895500004, -1...","POLYGON ((-101.846779221 31.47133973000007, -1...","POLYGON ((-100.61244462 32.26034043800007, -10...","POLYGON ((-100.630021874 32.26067696900003, -1...","POLYGON ((-102.131643432 33.18576756900006, -1...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1443,"POLYGON ((-104.157346543 29.49696445400008, -1...","POLYGON ((-100.849915782 29.77158865700005, -1...","POLYGON ((-102.661276135 30.17013560000004, -1...","POLYGON ((-101.322794175 30.50526702500002, -1...","POLYGON ((-101.345530604 30.52991678300003, -1...","POLYGON ((-101.433134502 30.55497581100008, -1...","POLYGON ((-100.597151416 31.63269333800002, -1...",POLYGON ((-102.0776815299999 30.81583088300005...,"POLYGON ((-100.428606279 31.68115339300005, -1...","POLYGON ((-100.60032301 31.68147734600007, -10...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1445,POLYGON ((-101.2662757159999 33.68582644900005...,"POLYGON ((-101.416110621 33.70962470300003, -1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
tx_test1_df = tx_block_1_df['geom_list'].apply(pd.Series).stack().copy()

In [62]:
tx_test1_df.head()

13  0    POLYGON ((-106.191802222 31.88349144100005, -1...
    1    POLYGON ((-106.191373993 31.82376458800007, -1...
    2    POLYGON ((-106.116162882 31.88287412000005, -1...
14  0    POLYGON ((-106.251938357 31.82622162100006, -1...
    1    POLYGON ((-106.2006129869999 31.88353904400003...
dtype: object

In [57]:
tx_test2_df = tx_test1_df.reset_index()

tx_test2_df.head()

,level_0,level_1,0
0,13,0,"POLYGON ((-106.191802222 31.88349144100005, -1..."
1,13,1,"POLYGON ((-106.191373993 31.82376458800007, -1..."
2,13,2,"POLYGON ((-106.116162882 31.88287412000005, -1..."
3,14,0,"POLYGON ((-106.251938357 31.82622162100006, -1..."
4,14,1,POLYGON ((-106.2006129869999 31.88353904400003...


In [58]:
tx_test2_df.index

RangeIndex(start=0, stop=5849, step=1)

## Method 2 ( I think this is it ! )

In [63]:
tx_block_2_df = tx_blocks_df.copy()
tx_block_2_df['geom_type'] = tx_block_1_df.geometry.geom_type


In [64]:
tx_block_2_df = tx_block_2_df.loc[tx_blocks_df.geom_type == 'MultiPolygon', :].copy()
tx_block_2_df.head()

,Township,Block,geometry,geom_type
13,1,78,"MULTIPOLYGON (((-106.19180 31.88349, -106.2006...",MultiPolygon
14,1,79,"MULTIPOLYGON (((-106.25194 31.82622, -106.2517...",MultiPolygon
16,1,81,"MULTIPOLYGON (((-106.42302 31.86312, -106.4232...",MultiPolygon
17,1,82,"MULTIPOLYGON (((-106.52890 31.91314, -106.5359...",MultiPolygon
18,10,48,"MULTIPOLYGON (((-102.56688 30.25972, -102.5721...",MultiPolygon


In [65]:
df_grantor = pd.DataFrame(tx_block_1_df['geom_list'].values.tolist()
                          , index=tx_block_2_df.index)



,0,1,2,3,4,5,6,7,8,9,...,582,583,584,585,586,587,588,589,590,591
13,"POLYGON ((-106.191802222 31.88349144100005, -1...","POLYGON ((-106.191373993 31.82376458800007, -1...","POLYGON ((-106.116162882 31.88287412000005, -1...",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
14,"POLYGON ((-106.251938357 31.82622162100006, -1...",POLYGON ((-106.2006129869999 31.88353904400003...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
16,POLYGON ((-106.4230167869999 31.86312426500007...,POLYGON ((-106.4742752929999 31.88429779100005...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
17,"POLYGON ((-106.528903858 31.91313742100004, -1...","POLYGON ((-106.528738027 31.98540340300008, -1...",POLYGON ((-106.5116119369999 31.95673094900008...,POLYGON ((-106.5287722959999 31.92766900600003...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
18,"POLYGON ((-102.566882766 30.25971651200007, -1...","POLYGON ((-103.16912323 30.83711843300006, -10...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [66]:
df_grantor['id'] = df_grantor.index


,0,1,2,3,4,5,6,7,8,9,...,583,584,585,586,587,588,589,590,591,id
13,"POLYGON ((-106.191802222 31.88349144100005, -1...","POLYGON ((-106.191373993 31.82376458800007, -1...","POLYGON ((-106.116162882 31.88287412000005, -1...",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,13
14,"POLYGON ((-106.251938357 31.82622162100006, -1...",POLYGON ((-106.2006129869999 31.88353904400003...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,14
16,POLYGON ((-106.4230167869999 31.86312426500007...,POLYGON ((-106.4742752929999 31.88429779100005...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,16
17,"POLYGON ((-106.528903858 31.91313742100004, -1...","POLYGON ((-106.528738027 31.98540340300008, -1...",POLYGON ((-106.5116119369999 31.95673094900008...,POLYGON ((-106.5287722959999 31.92766900600003...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,17
18,"POLYGON ((-102.566882766 30.25971651200007, -1...","POLYGON ((-103.16912323 30.83711843300006, -10...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,18


In [69]:
# Melt (columns to rows)
grantor = df_grantor.melt(
            id_vars='id'
            , value_vars=None
            #, var_name='grantor_var'
            , value_name='Split_Polygon'
        ).copy()

grantor.head()

,id,variable,Split_Polygon
0,13,0,"POLYGON ((-106.191802222 31.88349144100005, -1..."
1,14,0,"POLYGON ((-106.251938357 31.82622162100006, -1..."
2,16,0,POLYGON ((-106.4230167869999 31.86312426500007...
3,17,0,"POLYGON ((-106.528903858 31.91313742100004, -1..."
4,18,0,"POLYGON ((-102.566882766 30.25971651200007, -1..."


In [76]:
# Remove empty values
grantor.dropna(subset=['Split_Polygon'], inplace=True)
grantor.set_index('id', inplace=True)
grantor.shape

(5849, 2)

In [77]:
grantor.head()

,variable,Split_Polygon
id,,
13,0,"POLYGON ((-106.191802222 31.88349144100005, -1..."
14,0,"POLYGON ((-106.251938357 31.82622162100006, -1..."
16,0,POLYGON ((-106.4230167869999 31.86312426500007...
17,0,"POLYGON ((-106.528903858 31.91313742100004, -1..."
18,0,"POLYGON ((-102.566882766 30.25971651200007, -1..."


In [78]:
tx_block_2_df.head()

,Township,Block,geometry,geom_type
13,1,78,"MULTIPOLYGON (((-106.19180 31.88349, -106.2006...",MultiPolygon
14,1,79,"MULTIPOLYGON (((-106.25194 31.82622, -106.2517...",MultiPolygon
16,1,81,"MULTIPOLYGON (((-106.42302 31.86312, -106.4232...",MultiPolygon
17,1,82,"MULTIPOLYGON (((-106.52890 31.91314, -106.5359...",MultiPolygon
18,10,48,"MULTIPOLYGON (((-102.56688 30.25972, -102.5721...",MultiPolygon


In [80]:
df_join = tx_block_2_df.join(grantor, how='left')
df_join.head()

,Township,Block,geometry,geom_type,variable,Split_Polygon
13,1,78,"MULTIPOLYGON (((-106.19180 31.88349, -106.2006...",MultiPolygon,0,"POLYGON ((-106.191802222 31.88349144100005, -1..."
13,1,78,"MULTIPOLYGON (((-106.19180 31.88349, -106.2006...",MultiPolygon,1,"POLYGON ((-106.191373993 31.82376458800007, -1..."
13,1,78,"MULTIPOLYGON (((-106.19180 31.88349, -106.2006...",MultiPolygon,2,"POLYGON ((-106.116162882 31.88287412000005, -1..."
14,1,79,"MULTIPOLYGON (((-106.25194 31.82622, -106.2517...",MultiPolygon,0,"POLYGON ((-106.251938357 31.82622162100006, -1..."
14,1,79,"MULTIPOLYGON (((-106.25194 31.82622, -106.2517...",MultiPolygon,1,POLYGON ((-106.2006129869999 31.88353904400003...


In [81]:
df_join.shape

(5849, 6)